In [2]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import KY_utils.KY_baseline_functions as base

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\KY Model


In [3]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['drug_six_month'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['drug_six_month'].values

### Model

In [4]:
#### Logistic
c = [1e-4, 1e-3, 0.01, 0.1, 1]
logistic_summary = base.Logistic(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### Lasso
c = [1e-4, 1e-3, 0.01, 0.1, 1]
lasso_summary = base.Lasso(KY_X, KY_Y, FL_X, FL_Y, C=c,seed=816)

#### LinearSVM
c = [1e-4, 1e-3, 0.01, 0.1, 1]
svm_summary = base.LinearSVM(KY_X, KY_Y, FL_X, FL_Y, C=c,seed=816)

#### Random Forest
n_estimators =  [100,150,200]
depth = [7,8,9]
rf_summary = base.RF(KY_X, KY_Y, FL_X, FL_Y, 
                     depth=depth, 
                     estimators=n_estimators, 
                     seed=816)

#### XGBoost
learning_rate = [0.1]
depth = [4,5,6]
n_estimators = [100, 150]
xgb_summary = base.XGB(KY_X, KY_Y, FL_X, FL_Y,
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators,
                       seed=816)

#### save results
summary_drug6_ky_model = {"Logistic": logistic_summary,
                          "Lasso": lasso_summary,
                          "LinearSVM": svm_summary,
                          "RF": rf_summary,
                          "XGBoost": xgb_summary}

Stored 'summary_drug6_ky_model' (dict)


In [5]:
results = []
for model_name, model_summary in summary_drug6_ky_model.items():
    results.append([model_name, 
                    model_summary['best_auc'], 
                    model_summary['auc_diff'], 
                    model_summary['FL_score']])
results

[['Logistic', 0.7300883432509359, 0.0020886316305631647, 0.5939669782953365],
 ['Lasso', 0.7300939655271755, 0.0020985554825391217, 0.5941924990432453],
 ['LinearSVM', 0.7329589722483453, 0.0016251174982527816, 0.5858413974085617],
 ['RF', 0.7401599900410454, 0.012509276298722338, 0.568278114919906],
 ['XGBoost', 0.7540042440095361, 0.013656516414662478, 0.5921354764638347]]

In [6]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\baselines\\Six Month\\"
results = [["Drug",             
            round(logistic_summary['FL_score'],3), 
            round(lasso_summary['FL_score'], 3), 
            round(svm_summary['FL_score'],3), 
            round(rf_summary['FL_score'],3), 
            round(xgb_summary['FL_score'],3)]]
with open(path + 'Six Month KY_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)